In [2]:
import pandas as pd

In [62]:
def print_latex_table(df, caption, lines=True):
        print('\\begin{table}[H]')
        print('\\centering')
        headers = list(df)
        tabular_def = 'c|' + 'c'*(len(headers)-1)

        print('\\scriptsize{')
        print('\\begin{tabular}{' + tabular_def + '}')

        print('\\hline {}\\\\'.format(' & '.join(headers)))
        print('\\hline')
        for idx, row in df.iterrows():
            values = []
            for value in row.values:
                if type(value) == str:
                    values.append(value)
                elif float(value).is_integer():
                    values.append(str(int(value)))
                else:
                    values.append('{:.2f}'.format(value))
            print('{}\\\\'.format(' & '.join(values)))

        print('\\end{tabular}')
        print('}')
        print('\\caption{' + caption + '}')
        print('\\end{table}')
        print('%')

In [64]:
experiment = '3_1_full_system_write'
wl = 'set'
workload = '1-0'

m_m_1_df = pd.DataFrame()

df = pd.read_csv('{}/{}/{}_plot_data.csv'.format(experiment, workload, data_source))

df['arrival_rate'] = df['avg_set_tp_s']
service_rates = df.groupby('num_workers')['max_set_tp_s'].max()
service_rate_dfs = []
for num_worker in df['num_workers'].unique():
    mask = df.loc[:,'num_workers'] == num_worker
    df.loc[mask, 'service_rate'] = service_rates[num_worker]

df['traffic_intensity'] = df['arrival_rate']/df['service_rate']
df['avg_num_jobs_in_system'] = df['traffic_intensity']/(1 - df['traffic_intensity'])
df['avg_num_jobs_in_queue'] = df['traffic_intensity']*df['traffic_intensity']/(1 - df['traffic_intensity'])/(1 - df['traffic_intensity'])
df['avg_response_time'] = ((1/df['service_rate'])/(1-df['traffic_intensity']))*1000
df['avg_waiting_time'] = df['traffic_intensity']*df['avg_response_time']

for num_worker in df['num_workers'].unique():
    temp_df = pd.DataFrame()
    temp_df['Metric'] = [
        'Arrival Rate $\\lambda$',
        'Service Rate $\\mu$',
        'Traffic Intensity $\\rho$',
        'Avg. \\# of jobs in system',
        'Avg. \\# of jobs in queue',
        'Avg. Response Time',
        'Avg. Waiting Time']
    for num_clients in df['num_clients'].unique():
        select_list = [
            'arrival_rate',
            'service_rate',
            'traffic_intensity',
            'avg_num_jobs_in_system',
            'avg_num_jobs_in_queue',
            'avg_response_time',
            'avg_waiting_time'
        ]
        values = df[(df.num_clients == num_clients) & (df.num_workers == num_worker)][select_list].values[0]
        temp_df['C = {}'.format(num_clients)] = values
    print_latex_table(temp_df, 'Metrics for {} worker threads'.format(num_worker))

\begin{table}[H]
\centering
\scriptsize{
\begin{tabular}{c|ccccccc}
\hline Metric & C = 6 & C = 12 & C = 24 & C = 48 & C = 96 & C = 192 & C = 288\\
\hline
Arrival Rate $\lambda$ & 2869.38 & 5586.12 & 8290.70 & 8920.46 & 8925.23 & 8877.68 & 9059.10\\
Service Rate $\mu$ & 9800 & 9800 & 9800 & 9800 & 9800 & 9800 & 9800\\
Traffic Intensity $\rho$ & 0.29 & 0.57 & 0.85 & 0.91 & 0.91 & 0.91 & 0.92\\
Avg. \# of jobs in system & 0.41 & 1.33 & 5.49 & 10.14 & 10.20 & 9.63 & 12.23\\
Avg. \# of jobs in queue & 0.17 & 1.76 & 30.17 & 102.86 & 104.10 & 92.65 & 149.50\\
Avg. Response Time & 0.14 & 0.24 & 0.66 & 1.14 & 1.14 & 1.08 & 1.35\\
Avg. Waiting Time & 0.04 & 0.14 & 0.56 & 1.03 & 1.04 & 0.98 & 1.25\\
\end{tabular}
}
\caption{Metrics for 8 worker threads}
\end{table}
%
\begin{table}[H]
\centering
\scriptsize{
\begin{tabular}{c|ccccccc}
\hline Metric & C = 6 & C = 12 & C = 24 & C = 48 & C = 96 & C = 192 & C = 288\\
\hline
Arrival Rate $\lambda$ & 2798.44 & 5442.61 & 8553.49 & 9920.89 & 10265.37 & 1